In [1]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [2]:
from tensorflow import keras

In [3]:
tf.config.list_physical_devices('GPU')

[]

In [4]:
tf.device('/device:GPU:0')

In [5]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
df = pd.read_csv("/content/drive/MyDrive/Crypto/btcusdt_15m.csv")

In [ ]:
def data_load():
  df = pd.read_csv("/content/drive/MyDrive/Crypto/btcusdt_15m.csv")
  features = df.drop("datetime",axis=1)

  train_features = features
  features_scaler = MinMaxScaler(feature_range=(0, 1))
  train_features_normalized = features_scaler.fit_transform(train_features)

  trainX = []
  trainY = []

  n_future = 1
  n_past = 70

  for i in range(n_past, len(train_features_normalized) - n_future +1):
    trainX.append(train_features_normalized[i - n_past:i, 0:df.shape[1]])
    trainY.append(train_features_normalized[i + n_future - 1:i + n_future,3])

  return trainX[-n_past:]

In [ ]:
def forecast(model, n):
  future_predictions = []  ## list of future values
  current_batch = data_load()
  for _ in range(14):    ## iterating the same for 14 days future forecast

      # get the prediction value for the first batch
      current_pred = model.predict(current_batch)

      # print("Current_pred: {}".format(current_pred))
      # append the prediction into the array
      future_predictions.append(current_pred[0][0])

      # use the prediction to update the batch and remove the first value
      current_batch= np.delete(current_batch,0,axis=1)
      current_pred = np.repeat(current_pred, 5, axis=1)
      current_pred = current_pred.reshape((1,1,5))
      current_batch = np.append(current_batch,current_pred,axis=1)

  return future_predictions

In [ ]:
def add(date, open, high, low, close, volume):
  df = pd.read_csv("/content/drive/MyDrive/Crypto/btcusdt_15m.csv")

  # Create a new row with the values you want to add
  new_row = {'datetime' : date, 'open': open, 'high': high, 'low': low, 'close': close, 'volume': volume}

  # Add the new row to the DataFrame
  df = df.append(new_row, ignore_index=True)

  df.to_csv("/content/drive/MyDrive/Crypto/btcusdt_15m.csv", index = False)


In [ ]:
import tensorflow as tf

# Load the model from a file
model = tf.keras.models.load_model('/content/drive/MyDrive/Crypto/model.h5')
